In [1]:
import os
import pickle
from bs4 import BeautifulSoup
JAVA_SO_POSTS_STORE_PATH = 'C:/workspace/SOworkspace/data/so_posts/java/'
post_file_names = os.listdir(JAVA_SO_POSTS_STORE_PATH)
post_file_names[0:1]

['posts_0.pkl']

In [45]:
'''
从已经提取的句子里找点case做一下处理尝试
下面是选取的典型句子sample
'''
sample_text = '''<p>As a part of a project I want to make a table with different type of object for each cell but I can't understand how to make every cell customized ... I found <a href=\"https://docs.oracle.com/javase/tutorial/uiswing/components/table.html\" rel=\"nofollow noreferrer\">here</a> how to set a column which has a type of object like String, Bool, Slider ... but for each Cell I don't know!</p>&#xA;&#xA;<p>This picture is from properties part of <a href=\"http://www.formdev.com/\" rel=\"nofollow noreferrer\">jformDesigner</a> software and I want something like this table :</p>&#xA;&#xA;<p><a href=\"https://i.stack.imgur.com/ZL8GO.jpg\" rel=\"nofollow noreferrer\"><img src=\"https://i.stack.imgur.com/ZL8GO.jpg\" alt=\"enter image description here\"></a></p>&#xA;'''
    

In [46]:
soup = BeautifulSoup(sample_text, 'lxml')
soup.text

"As a part of a project I want to make a table with different type of object for each cell but I can't understand how to make every cell customized ... I found here how to set a column which has a type of object like String, Bool, Slider ... but for each Cell I don't know!\nThis picture is from properties part of jformDesigner software and I want something like this table :\n\n"

In [47]:
for pre in soup.find_all('pre'):
    pre.extract()
soup.text

"As a part of a project I want to make a table with different type of object for each cell but I can't understand how to make every cell customized ... I found here how to set a column which has a type of object like String, Bool, Slider ... but for each Cell I don't know!\nThis picture is from properties part of jformDesigner software and I want something like this table :\n\n"

In [48]:
soup

<html><body><p>As a part of a project I want to make a table with different type of object for each cell but I can't understand how to make every cell customized ... I found <a href="https://docs.oracle.com/javase/tutorial/uiswing/components/table.html" rel="nofollow noreferrer">here</a> how to set a column which has a type of object like String, Bool, Slider ... but for each Cell I don't know!</p>
<p>This picture is from properties part of <a href="http://www.formdev.com/" rel="nofollow noreferrer">jformDesigner</a> software and I want something like this table :</p>
<p><a href="https://i.stack.imgur.com/ZL8GO.jpg" rel="nofollow noreferrer"><img alt="enter image description here" src="https://i.stack.imgur.com/ZL8GO.jpg"/></a></p>
</body></html>

In [49]:
a_texts = [a.text for a in soup.find_all('a') if len(a.text) > 1]
a_texts

['here', 'jformDesigner']

In [50]:
import re
lines = re.split(r'(\s*(\.|:|\?|!)\s+|\n)+', soup.text)
lines

["As a part of a project I want to make a table with different type of object for each cell but I can't understand how to make every cell customized ..",
 '. ',
 '.',
 'I found here how to set a column which has a type of object like String, Bool, Slider ..',
 '. ',
 '.',
 "but for each Cell I don't know",
 '!\n',
 '!',
 'This picture is from properties part of jformDesigner software and I want something like this table',
 ' :\n\n',
 ':',
 '']

In [51]:
candidate_lines = [line for line in lines if any([t for t in a_texts if t in line])]
candidate_lines

['I found here how to set a column which has a type of object like String, Bool, Slider ..',
 'This picture is from properties part of jformDesigner software and I want something like this table']

In [10]:
def get_all_indexes(sub : str, s : str):
    ret = []
    start_from_index = 0
    while start_from_index < len(s):
        sub_begin_index = s.find(sub,start_from_index)
        if sub_begin_index == -1:
            break
        sub_end_index = sub_begin_index + len(sub) - 1
        ret.append([sub_begin_index, sub_end_index])
        start_from_index = sub_end_index + 1
    return ret

get_all_indexes('diamond operator' , '''It's called the diamond operator. It was introduced in Java 1.7.''')

[[16, 31]]

In [11]:
trainset = []
for candidate in candidate_lines:
    train_data = {
        "text" : candidate,
        "label" : {
            "api" : {}
        }
    }
    labels = {}
    for a_text in a_texts:
        if a_text in candidate:
            labels[a_text] = get_all_indexes(a_text, candidate)
    train_data["label"]["api"] = labels
    trainset.append(train_data)
trainset

[{'text': "It's called the diamond operator. It was introduced in Java 1.7.",
  'label': {'api': {'diamond operator': [[16, 31]]}}}]

In [12]:
import json
json_str = json.dumps(trainset[0], ensure_ascii=False)

In [13]:
json_str
"""
训练集生成尝试完毕，上述代码将被总结并加入EUREKA-2
"""

'{"text": "It\'s called the diamond operator. It was introduced in Java 1.7.", "label": {"api": {"diamond operator": [[16, 31]]}}}'